In [ ]:

!apt-get update
!apt-get install -y wget unzip libvulkan1

!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f -y

!apt-get install -y xvfb

!pip install selenium chromedriver-autoinstaller
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,320 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,159 kB]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hi

In [ ]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup


chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# WebDriver
driver = webdriver.Chrome(options=chrome_options)
driver.set_page_load_timeout(60)
driver.implicitly_wait(10)
wait = WebDriverWait(driver, 30)

# handle login
def login(driver):
    try:
        # paywall has login btn
        login_button = driver.find_element(By.CLASS_NAME, 'c-paywall__btn-login')
        login_button.click()

        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, 'Username')))

        # login credentials
        email_input = driver.find_element(By.ID, 'Username')
        email_input.send_keys('grp613@alumni.ku.dk')
        password_input = driver.find_element(By.NAME, 'Password')
        password_input.send_keys('xxxx')

        # submit the form
        login_submit = driver.find_element(By.CSS_SELECTOR, 'button[type="submit"]')
        login_submit.click()

        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'c-article-text-container')))
    except TimeoutException:
        print("Timeout while trying to login.")
    except NoSuchElementException:
        print("Unable to locate an element during login.")

def accept_cookies(driver):
    try:
        cookies_button = driver.find_element(By.ID, 'CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll')
        cookies_button.click()
    except NoSuchElementException:
        pass

def get_article_links():
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = soup.find_all('article', class_='grid grid-cols-8 space-y-0.5 border-b border-black px-1 py-3')
    article_links = []
    for article in articles:
        a_tag = article.find('a', class_='group col-span-8 grid grid-cols-8 pt-1.5')
        if a_tag:
            href = a_tag['href']
            full_url = "https://finans.dk" + href
            article_links.append(full_url)
    return article_links


def scrape_article(article_url):
    print(f"Scraping article: {article_url}")
    try:
        driver.get(article_url)
        time.sleep(5)
        accept_cookies(driver)
    except TimeoutException:
        print(f"Timeout when loading article: {article_url}")
        return {'url': article_url, 'header': 'Timeout', 'subheader': '', 'date': '', 'content': 'Timeout'}

    try:
        paywall = driver.find_element(By.CLASS_NAME, 'c-paywall__title')
        if paywall:
            login(driver)
    except NoSuchElementException:
        pass

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    try:
        header = soup.find('h1', class_='c-article-top-info__title').get_text()
    except:
        header = 'No header'

    try:
        subheader = soup.find('p', class_='c-article-top-info__description').get_text()
    except:
        subheader = 'No subheader'

    try:
        date = soup.find('time')['datetime']
    except:
        date = 'No date'

    try:
        paragraphs = driver.find_elements(By.CSS_SELECTOR, '.c-article-text-container p')
        content = " ".join([p.text for p in paragraphs])
    except NoSuchElementException:
        content = "No content"

    return {
        'url': article_url,
        'header': header,
        'subheader': subheader,
        'date': date,
        'content': content,
        'search_term': term
    }

def scrape_articles(search_term):
    page_num = 1
    all_articles = []

    while True:
        search_url = f"https://finans.dk/soeg?term={search_term}&page={page_num}"
        try:
            driver.get(search_url)
            time.sleep(5)
            accept_cookies(driver)
        except TimeoutException:
            print(f"Timeout when loading search page {page_num} for term {search_term}")
            break

        article_links = get_article_links()

        if not article_links:
            print(f"No more articles found on page {page_num} for term '{search_term}'.")
            break

        for article_url in article_links:
            article_data = scrape_article(article_url)
            all_articles.append(article_data)

        print(f"Scraped page {page_num}...")

        page_num += 1

    return all_articles


search_terms = ['ansigtsgenkendelse', 'facial recognition', 'face recognition', 'ansigtsaftryk', 'ansigtsscan',
                'masseovervågning ansigt', 'masseovervågning kamera', 'ansigtskamera', 'overvågningssamfund ansigt',
                'overvågningssamfund kamera', 'sikkerhedspakke overvågning', 'tryghedsskabende kamera']

all_scraped_articles = []
for term in search_terms:
    scraped_articles = scrape_articles(term)
    all_scraped_articles.extend(scraped_articles)

df = pd.DataFrame(all_scraped_articles)
df.to_csv('finans_scraped_articles.csv', index=False)

driver.quit()


Scraping article: https://finans.dk/indland/ECE17389092/ansigtsgenkendelse-er-paa-vej-i-danmark-central-direktoer-kommer-med-en-klar-advarsel/
Scraping article: https://finans.dk/international/asien/ECE16449619/lufthavn-i-singapore-erstatter-pas-med-ansigtsgenkendelse/
Scraping article: https://finans.dk/erhverv/ECE17463765/dansk-techselskab-er-blevet-solgt-for-et-milliardbeloeb/
Scraping article: https://finans.dk/tech/ECE16677163/milestone-systems-glaeder-sig-over-den-store-lovpakke-om-kunstig-intelligens-det-giver-mindre-tyngende-ansvar/
Scraping article: https://finans.dk/erhverv/ECE16485441/fogh-regulering-af-ai-skal-bygge-paa-demokratiske-vaerdier/
Scraping article: https://finans.dk/tech/ECE17084257/ukendt-dansk-techselskab-har-rejst-over-450-mio-kr-18-aars-tilloeb-skal-nu-omsaettes-til-global-sejrsgang/
Scraping article: https://finans.dk/debat/ECE16822021/store-omvaeltninger-venter-finanssektoren-frem-mod-2035/
Scraping article: https://finans.dk/debat/ECE16842589/pas-paa-hack

In [ ]:
df

,url,header,subheader,date,content,search_term
0,https://finans.dk/indland/ECE17389092/ansigtsg...,Ansigtsgenkendelse er på vej i Danmark – centr...,Thomas Jensen har længe presset på for en dans...,2024-09-01 08:00:00.000+0200,Der er behov for at sætte klare hegnspæle for ...,ansigtsgenkendelse
1,https://finans.dk/international/asien/ECE16449...,Lufthavn i Singapore erstatter pas med ansigts...,Fra 2024 vil man i Changi Lufthavn i Singapore...,2023-09-21 06:17:09.000+0200,Fra næste år bliver det muligt at rejse gennem...,ansigtsgenkendelse
2,https://finans.dk/erhverv/ECE17463765/dansk-te...,Dansk tech-selskab er blevet solgt for et mill...,NIL Technology med speciale i nanoteknologi er...,2024-09-20 06:55:00.000+0200,Det danske tech-selskab NIL Technology (NILT) ...,ansigtsgenkendelse
3,https://finans.dk/tech/ECE16677163/milestone-s...,Milestone Systems glæder sig over den store lo...,Det bliver op til virksomhederne selv at udøve...,2023-12-12 10:00:00.000+0100,Efter 36 timers forhandling i løbet af tre dag...,ansigtsgenkendelse
4,https://finans.dk/erhverv/ECE16485441/fogh-reg...,Fogh: Regulering af AI skal bygge på demokrati...,Rasmussen Global og softwarevirksomheden Miles...,2023-10-04 18:17:00.000+0200,"Anders Fogh Rasmussen mener, at regulering af ...",ansigtsgenkendelse
...,...,...,...,...,...,...
265,https://finans.dk/erhverv/ECE10751537/big-brot...,Big Brother på kinesisk: Beijings overvågnings...,Det er svært at gemme sig for et overvågningss...,2018-07-14 11:00:00.000+0200,TOKYO – Træder du ind i receptionen i det kine...,overvågningssamfund kamera
266,https://finans.dk/politik/ECE11671976/taenketa...,Tænketank: Underligt at lægge op til øget over...,"Det er bemærkelsesværdigt, at statsministeren ...",2019-10-08 22:53:18.000+0200,"Det er besynderlig timing, når statsminister M...",sikkerhedspakke overvågning
267,https://finans.dk/erhverv/ECE10781137/facebook...,Facebook bruger over 100 millioner på chefens ...,Det kommer til at koste Facebook over 110 mio....,2018-08-02 20:32:50.000+0200,Det er ikke gratis at være en af verdens mest ...,sikkerhedspakke overvågning
268,https://finans.dk/politik/ECE17356882/venstre-...,Venstre vil have økonomisk kriminelle til at b...,"Venstre mener, at dømte for alvorlig økonomisk...",2024-08-16 06:12:40.000+0200,Hvis man bliver dømt for alvorlig økonomisk kr...,tryghedsskabende kamera


In [ ]:
df = df.drop_duplicates(subset=['url', 'date'], keep='first')
df

,url,header,subheader,date,content,search_term
0,https://finans.dk/indland/ECE17389092/ansigtsg...,Ansigtsgenkendelse er på vej i Danmark – centr...,Thomas Jensen har længe presset på for en dans...,2024-09-01 08:00:00.000+0200,Der er behov for at sætte klare hegnspæle for ...,ansigtsgenkendelse
1,https://finans.dk/international/asien/ECE16449...,Lufthavn i Singapore erstatter pas med ansigts...,Fra 2024 vil man i Changi Lufthavn i Singapore...,2023-09-21 06:17:09.000+0200,Fra næste år bliver det muligt at rejse gennem...,ansigtsgenkendelse
2,https://finans.dk/erhverv/ECE17463765/dansk-te...,Dansk tech-selskab er blevet solgt for et mill...,NIL Technology med speciale i nanoteknologi er...,2024-09-20 06:55:00.000+0200,Det danske tech-selskab NIL Technology (NILT) ...,ansigtsgenkendelse
3,https://finans.dk/tech/ECE16677163/milestone-s...,Milestone Systems glæder sig over den store lo...,Det bliver op til virksomhederne selv at udøve...,2023-12-12 10:00:00.000+0100,Efter 36 timers forhandling i løbet af tre dag...,ansigtsgenkendelse
4,https://finans.dk/erhverv/ECE16485441/fogh-reg...,Fogh: Regulering af AI skal bygge på demokrati...,Rasmussen Global og softwarevirksomheden Miles...,2023-10-04 18:17:00.000+0200,"Anders Fogh Rasmussen mener, at regulering af ...",ansigtsgenkendelse
...,...,...,...,...,...,...
262,https://finans.dk/indland/ECE12627849/danmark-...,Danmark skal være mere standhaftig over for de...,Efter kinesisk vrede over et møde med Dalai La...,2020-12-19 20:45:00.000+0100,Angreb på demokratiforkæmpere i Hongkong samt ...,masseovervågning ansigt
266,https://finans.dk/politik/ECE11671976/taenketa...,Tænketank: Underligt at lægge op til øget over...,"Det er bemærkelsesværdigt, at statsministeren ...",2019-10-08 22:53:18.000+0200,"Det er besynderlig timing, når statsminister M...",sikkerhedspakke overvågning
267,https://finans.dk/erhverv/ECE10781137/facebook...,Facebook bruger over 100 millioner på chefens ...,Det kommer til at koste Facebook over 110 mio....,2018-08-02 20:32:50.000+0200,Det er ikke gratis at være en af verdens mest ...,sikkerhedspakke overvågning
268,https://finans.dk/politik/ECE17356882/venstre-...,Venstre vil have økonomisk kriminelle til at b...,"Venstre mener, at dømte for alvorlig økonomisk...",2024-08-16 06:12:40.000+0200,Hvis man bliver dømt for alvorlig økonomisk kr...,tryghedsskabende kamera


In [ ]:
from google.colab import files

df.to_csv('finans_scraped_articles.csv')
files.download('finans_scraped_articles.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>